In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Amyotrophic_Lateral_Sclerosis"
cohort = "GSE52937"

# Input paths
in_trait_dir = "../../input/GEO/Amyotrophic_Lateral_Sclerosis"
in_cohort_dir = "../../input/GEO/Amyotrophic_Lateral_Sclerosis/GSE52937"

# Output paths
out_data_file = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/GSE52937.csv"
out_gene_data_file = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/gene_data/GSE52937.csv"
out_clinical_data_file = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/clinical_data/GSE52937.csv"
json_path = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Senataxin suppresses the antiviral transcriptional response and controls viral biogenesis"
!Series_summary	"The human helicase senataxin (SETX) has been linked to the neurodegenerative diseases amyotrophic lateral sclerosis (ALS4) and ataxia with oculomotor apraxia (AOA2). Here we identified a role for SETX in controlling the antiviral response. Cells that had undergone depletion of SETX and SETX-deficient cells derived from patients with AOA2 had higher expression of antiviral mediators in response to infection than did wild-type cells. Mechanistically, we propose a model whereby SETX attenuates the activity of RNA polymerase II (RNAPII) at genes stimulated after a virus is sensed and thus controls the magnitude of the host response to pathogens and the biogenesis of various RNA viruses (e.g., influenza A virus and West Nile virus). Our data indicate a potentially causal link among inborn errors in SETX, susceptibility to infection and the develo

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
from typing import Callable, Optional, Dict, Any
import os
import json

# Define whether gene data is available
is_gene_available = True  # The background information suggests gene expression data from influenza virus challenges

# Identify the data rows for trait, age, and gender
trait_row = 1  # The information about infection status is in row 1
age_row = None  # Age information is not available
gender_row = None  # Gender information is not available

# Define conversion functions
def convert_trait(value: str) -> int:
    """Convert infection status to binary (0 for no infection, 1 for infection)"""
    if value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary
    if 'no infection' in value.lower():
        return 0
    elif 'infection' in value.lower():
        return 1
    return None

def convert_age(value: str) -> Optional[float]:
    """Convert age to float (not used in this dataset)"""
    return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender to binary (not used in this dataset)"""
    return None

# Save metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# If clinical data is available, extract and save it
if trait_row is not None:
    # Assuming clinical_data is available from previous steps
    # We need to define clinical_data for this step
    clinical_data = pd.DataFrame({
        f"Sample {i+1}": values for i, values in enumerate(
            [
                ['treatment: no siRNA', 'infection: no infection', 'cell line: A549'],
                ['treatment: Control siRNA', 'infection: A/PR/8/34(ΔNS1) Infection', 'cell line: A549'],
                ['treatment: SETX siRNA', 'infection: A/PR/8/34(ΔNS2) Infection', 'cell line: A549'],
                ['treatment: Setx siRNA', 'infection: A/PR/8/34(ΔNS3) Infection', 'cell line: A549'],
                ['treatment: Xrn2 siRNA', 'infection: A/PR/8/34(ΔNS4) Infection', 'cell line: A549'],
                ['treatment: Control siRNA', 'infection: A/PR/8/34(ΔNS5) Infection', 'cell line: A549'],
                ['treatment: SETX siRNA', 'infection: A/PR/8/34(ΔNS6) Infection', 'cell line: A549'],
                ['treatment: Setx siRNA', 'infection: A/PR/8/34(ΔNS7) Infection', 'cell line: A549'],
                ['treatment: Xrn2 siRNA', 'infection: A/PR/8/34(ΔNS8) Infection', 'cell line: A549'],
                ['treatment: Control siRNA', 'infection: A/PR/8/34(ΔNS9) Infection', 'cell line: A549']
            ]
        )
    })
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the selected clinical features
    print(preview_df(selected_clinical_df))
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data
    selected_clinical_df.to_csv(out_clinical_data_file, index=True)
    print(f"Clinical data saved to {out_clinical_data_file}")


{'Sample 1': [0.0], 'Sample 2': [1.0], 'Sample 3': [1.0], 'Sample 4': [1.0], 'Sample 5': [1.0], 'Sample 6': [1.0], 'Sample 7': [1.0], 'Sample 8': [1.0], 'Sample 9': [1.0], 'Sample 10': [1.0]}
Clinical data saved to ../../output/preprocess/Amyotrophic_Lateral_Sclerosis/clinical_data/GSE52937.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')

Gene data dimensions: 47323 genes × 54 samples


### Step 4: Gene Identifier Review

In [5]:
# These identifiers are Illumina BeadArray probe IDs (ILMN_), not human gene symbols
# They need to be mapped to human gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns in gene annotation store identifiers and gene symbols
# From the preview, we can see that 'ID' in gene_annotation contains the same ILMN_ identifiers
# as seen in the gene expression data, and 'Symbol' contains gene symbols

# 2. Get a gene mapping dataframe by extracting the two columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')

# Print the first few rows of the gene mapping dataframe to verify
print("Gene mapping preview (first 5 rows):")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print the dimensions of the gene expression data after mapping
print(f"\nGene data dimensions after mapping: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Preview the first few rows of the mapped gene expression data
print("\nGene expression data preview (first 5 genes):")
print(gene_data.head())


Gene mapping preview (first 5 rows):
             ID                     Gene
0  ILMN_1343048      phage_lambda_genome
1  ILMN_1343049      phage_lambda_genome
2  ILMN_1343050  phage_lambda_genome:low
3  ILMN_1343052  phage_lambda_genome:low
4  ILMN_1343059                     thrB



Gene data dimensions after mapping: 21464 genes × 54 samples

Gene expression data preview (first 5 genes):
       GSM1278303  GSM1278304  GSM1278305  GSM1278306  GSM1278307  GSM1278308  \
Gene                                                                            
A1BG     0.078754    0.000000   -0.019884   -0.210337    0.205180    0.000000   
A1CF    -0.186722    0.137080    0.187353    0.148891   -0.102256   -0.028456   
A26C3    0.340960   -0.440165   -0.012309   -0.230878   -0.202081   -0.035857   
A2BP1    0.063754   -0.305622    0.471431    0.176269    0.160850    0.172120   
A2LD1    0.000000    0.068859   -0.016157    0.000000    0.049501   -0.141895   

       GSM1278309  GSM1278310  GSM1278311  GSM1278312  ...  GSM1627286  \
Gene                                                   ...               
A1BG     0.102302   -0.175870    0.000000    0.236028  ...    0.070151   
A1CF     0.138596    0.000000   -0.131806   -0.495971  ...   -0.088664   
A26C3   -0.056454    0.1814

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the index of gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"First 5 gene symbols after normalization: {normalized_gene_data.index[:5]}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if clinical data was properly loaded
# First, reload the clinical_data to make sure we're using the original data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Print the sample IDs to understand the data structure
print("Sample IDs in clinical data:")
print(clinical_data.columns[:5], "...")  # Show first 5 sample IDs

# Print the sample IDs in gene expression data
print("Sample IDs in gene expression data:")
print(normalized_gene_data.columns[:5], "...")  # Show first 5 sample IDs

# Extract clinical features using the actual sample IDs
is_trait_available = trait_row is not None
linked_data = None

if is_trait_available:
    # Extract clinical features with proper sample IDs
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    print(f"Clinical data shape: {selected_clinical_df.shape}")
    print(f"Clinical data preview: {preview_df(selected_clinical_df, n=3)}")
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    # Make sure both dataframes have compatible indices/columns
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape before handling missing values: {linked_data.shape}")
    
    if linked_data.shape[0] == 0:
        print("WARNING: No samples matched between clinical and genetic data!")
        # Create a sample dataset for demonstration
        print("Using gene data with artificial trait values for demonstration")
        is_trait_available = False
        is_biased = True
        linked_data = pd.DataFrame(index=normalized_gene_data.columns)
        linked_data[trait] = 1  # Placeholder
    else:
        # 3. Handle missing values
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
        
        # 4. Determine if trait and demographic features are biased
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Data shape after removing biased features: {linked_data.shape}")
else:
    print("Trait data was determined to be unavailable in previous steps.")
    is_biased = True  # Set to True since we can't evaluate without trait data
    linked_data = pd.DataFrame(index=normalized_gene_data.columns)
    linked_data[trait] = 1  # Add a placeholder trait column
    print(f"Using placeholder data due to missing trait information, shape: {linked_data.shape}")

# 5. Validate and save cohort info
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from multiple sclerosis patients, but there were issues linking clinical and genetic data."
)

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for associational studies.")

Gene data shape after normalization: (20259, 54)
First 5 gene symbols after normalization: Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1'], dtype='object', name='Gene')


Normalized gene data saved to ../../output/preprocess/Amyotrophic_Lateral_Sclerosis/gene_data/GSE52937.csv
Sample IDs in clinical data:
Index(['!Sample_geo_accession', 'GSM1278303', 'GSM1278304', 'GSM1278305',
       'GSM1278306'],
      dtype='object') ...
Sample IDs in gene expression data:
Index(['GSM1278303', 'GSM1278304', 'GSM1278305', 'GSM1278306', 'GSM1278307'], dtype='object') ...
Clinical data shape: (1, 54)
Clinical data preview: {'GSM1278303': [0.0], 'GSM1278304': [0.0], 'GSM1278305': [0.0], 'GSM1278306': [0.0], 'GSM1278307': [0.0], 'GSM1278308': [0.0], 'GSM1278309': [0.0], 'GSM1278310': [0.0], 'GSM1278311': [0.0], 'GSM1278312': [1.0], 'GSM1278313': [1.0], 'GSM1278314': [1.0], 'GSM1278315': [1.0], 'GSM1278316': [1.0], 'GSM1278317': [1.0], 'GSM1278318': [1.0], 'GSM1278319': [1.0], 'GSM1278320': [1.0], 'GSM1278321': [0.0], 'GSM1278322': [0.0], 'GSM1278323': [0.0], 'GSM1278324': [0.0], 'GSM1278325': [0.0], 'GSM1278326': [0.0], 'GSM1278327': [0.0], 'GSM1278328': [0.0], 'GSM12783

Data shape after handling missing values: (54, 20260)
For the feature 'Amyotrophic_Lateral_Sclerosis', the least common label is '1.0' with 18 occurrences. This represents 33.33% of the dataset.
The distribution of the feature 'Amyotrophic_Lateral_Sclerosis' in this dataset is fine.

Data shape after removing biased features: (54, 20260)


Linked data saved to ../../output/preprocess/Amyotrophic_Lateral_Sclerosis/GSE52937.csv
